In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
# !git pull
# 0.3860962

In [3]:
import tensorflow as tf
import malaya_speech.augmentation.waveform as augmentation
from malaya_speech.train.model import denoiser, stft
import malaya_speech
from glob import glob
import random
import numpy as np
import IPython.display as ipd

np.seterr(all='raise')






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [4]:
# files = glob('../youtube/clean-wav/*.wav')
# random.shuffle(files)
# len(files)

# noises = glob('../noise-44k/noise/*.wav') + glob('../noise-44k/clean-wav/*.wav')
# basses = glob('HHDS/Sources/**/*bass.wav', recursive = True)
# drums = glob('HHDS/Sources/**/*drums.wav', recursive = True)
# others = glob('HHDS/Sources/**/*other.wav', recursive = True)
# noises = noises + basses + drums + others
# random.shuffle(noises)

In [5]:
# def read_wav(f):
#     return malaya_speech.load(f, sr = 44100)


# def random_sampling(s, length):
#     return augmentation.random_sampling(s, sr = 44100, length = length)


# def combine_speakers(files, n = 5):
#     w_samples = random.sample(files, n)
#     w_samples = [
#         random_sampling(
#             read_wav(f)[0],
#             length = min(
#                 random.randint(20000 // n, 240_000 // n), 100_000 // n
#             ),
#         )
#         for f in w_samples
#     ]
#     y = [w_samples[0]]
#     left = w_samples[0].copy() * random.uniform(0.5, 1.0)
#     for i in range(1, n):

#         right = w_samples[i].copy() * random.uniform(0.5, 1.0)

#         overlap = random.uniform(0.01, 1.25)
#         left_len = int(overlap * len(left))

#         padded_right = np.pad(right, (left_len, 0))

#         if len(left) > len(padded_right):
#             padded_right = np.pad(
#                 padded_right, (0, len(left) - len(padded_right))
#             )
#         else:
#             left = np.pad(left, (0, len(padded_right) - len(left)))

#         y.append(padded_right)
#         left = left + padded_right
#     return left, y

# def random_amplitude(sample, low = 3, high = 5):
#     y_aug = sample.copy()
#     dyn_change = np.random.uniform(low = low, high = high)
#     y_aug = y_aug * dyn_change
#     return np.clip(y_aug, -1, 1)

# def random_amplitude_threshold(sample, low = 1, high = 2, threshold = 0.4):
#     y_aug = sample.copy()
#     y_aug = y_aug / (np.max(np.abs(y_aug)) + 1e-9)
#     dyn_change = np.random.uniform(low = low, high = high)
#     y_aug[np.abs(y_aug) >= threshold] = (
#         y_aug[np.abs(y_aug) >= threshold] * dyn_change
#     )
#     return np.clip(y_aug, -1, 1)


# def calc(signal, seed, add_uniform = False):
#     random.seed(seed)

#     choice = random.randint(0, 6)
#     print('choice', choice)
#     if choice == 0:

#         x = augmentation.sox_augment_high(
#             signal,
#             min_bass_gain = random.randint(25, 50),
#             reverberance = random.randint(0, 80),
#             hf_damping = 10,
#             room_scale = random.randint(0, 50),
#             negate = 1,
#         )
#     if choice == 1:
#         x = augmentation.sox_augment_high(
#             signal,
#             min_bass_gain = random.randint(25, 70),
#             reverberance = random.randint(0, 80),
#             hf_damping = 10,
#             room_scale = random.randint(0, 50),
#             negate = 0,
#         )
#     if choice == 2:
#         x = augmentation.sox_augment_low(
#             signal,
#             min_bass_gain = random.randint(5, 30),
#             reverberance = random.randint(0, 80),
#             hf_damping = 10,
#             room_scale = random.randint(0, 50),
#             negate = random.randint(0, 1),
#         )
#     if choice == 3:
#         x = augmentation.sox_augment_combine(
#             signal,
#             min_bass_gain_high = random.randint(25, 70),
#             min_bass_gain_low = random.randint(5, 30),
#             reverberance = random.randint(0, 80),
#             hf_damping = 10,
#             room_scale = random.randint(0, 90),
#         )
#     if choice == 4:
#         x = augmentation.sox_reverb(
#             signal,
#             reverberance = random.randint(10, 80),
#             hf_damping = 10,
#             room_scale = random.randint(10, 90),
#         )
#     if choice == 5:
#         x = random_amplitude_threshold(
#             signal, threshold = random.uniform(0.35, 0.8)
#         )

#     if choice == 6:
#         x = signal

#     if choice not in [5] and random.gauss(0.5, 0.14) >= 0.6:
#         x = random_amplitude_threshold(
#             x, low = 1.0, high = 2.0, threshold = random.uniform(0.6, 0.9)
#         )
    
#     if random.gauss(0.5, 0.14) > 0.6 and add_uniform:
#         x = augmentation.add_uniform_noise(
#             x, power = random.uniform(0.005, 0.015)
#         )

#     return x

In [6]:
# def parallel(f):
#     if random.gauss(0.5, 0.14) > 0.6:
#         s = random.sample(files, random.randint(2, 6))
#         y = combine_speakers(s, len(s))[0]
#     else:
#         y = random_sampling(
#             read_wav(f)[0], length = random.randint(30000, 100_000)
#         )
        
#     y = y / (np.max(np.abs(y)) + 1e-9)

#     seed = random.randint(0, 100_000_000)
#     x = calc(y, seed)
#     if random.gauss(0.5, 0.14) > 0.6:
#         print('add small noise')
#         n = combine_speakers(noises, random.randint(1, 20))[0]
#         n = calc(n, seed, True)
#         combined, noise = augmentation.add_noise(
#             x, n, factor = random.uniform(0.01, 0.1), return_noise = True
#         )
#     else:
#         x = x / (np.max(np.abs(x)) + 1e-9)
#         combined = x
#     noise = combined - y
#     return combined, y, noise

In [7]:
# sr = 44100
# combined, actual, noise = parallel(files[0])

In [8]:
# import pickle

# with open('s.pkl', 'wb') as fopen:
#     pickle.dump([combined, actual, noise], fopen)

In [9]:
import IPython.display as ipd

In [10]:
import pickle

with open('s.pkl', 'rb') as fopen:
    combined, actual, noise = pickle.load(fopen)

In [11]:
# ipd.Audio(noise[:sr *20], rate = new_sr)

In [12]:
# ipd.Audio(y_[:sr *20,0], rate = new_sr)

In [13]:
sr = 44100
combined = combined[: 20 * sr]
actual = actual[: 20 * sr]
noise = noise[: 20 * sr]
# combined = malaya_speech.resample(combined, sr, new_sr)
# actual = malaya_speech.resample(actual, sr, new_sr)
# noise = malaya_speech.resample(noise, sr, new_sr)

In [14]:
# len(combined)

In [15]:
# len(malaya_speech.resample(combined, sr, 22050))

In [16]:
# ipd.Audio(actual,
#           rate = sr)

In [17]:
# ipd.Audio(malaya_speech.resample(actual, sr, 16000), rate = 16000)

In [18]:
# ipd.Audio(malaya_speech.resample(malaya_speech.resample(actual, sr, 12000), 12000, sr),
#           rate = sr)

In [19]:
# l = malaya_speech.resample(malaya_speech.resample(actual, sr, 8000), 8000, sr)

In [20]:
# np.abs(actual - l).mean()

In [21]:
# ipd.Audio(malaya_speech.resample(actual, sr, 22050), rate = new_sr)

In [22]:
i = tf.placeholder(tf.float32, [None, 1])
y = tf.placeholder(tf.float32, [None, 1])
model = denoiser.Model(i, logging = True, normalize = True, stride = 4, 
                       partition_length = sr)
logits = model.logits
#logits = logits * i

Tensor("Pad_1:0", shape=(?, 44117, 1), dtype=float32)
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Tensor("concat_2:0", shape=(?, ?, 1), dtype=float32)
Tensor("concat_2:0", shape=(?, ?, 1), dtype=float32)
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use `tf.keras.layers.Conv1D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Tensor("sequential/conv1d_1/mul:0", shape=(?, ?, 48), dtype=float32)
Tensor("sequential_2/conv1d_4/mul:0", shape=(?, ?, 96), dtype=float32)
Tensor("sequential_4/conv1d_7/mul:0", shape=(?, ?, 192), dtype=float32)
Tensor("sequential_6/conv1d_10/mul:0", shape=(?, ?, 384), dtype=float32)
x Tensor("sequential_8/conv1d_13/mul:0", shape=(?, ?, 768), dtype=float32)
Tensor("dense/BiasAdd:0", shape=(?, ?, 768), dtype=float32)
Tensor("sequential_9/Conv1DTranspose/strided_slice:0", shape=(?, ?, 384), dtype=float32)
Tensor("sequential_7

In [23]:
stft_loss = stft.loss.MultiResolutionSTFT(factor_sc = 0.2, factor_mag = 0.2,
                                         fft_lengths = [1024, 2048, 512],
                                         frame_lengths = [600, 1200, 240],
                                         frame_steps = [50, 50, 50])
l1 = tf.reduce_mean(tf.abs(y - logits), axis = 0)
sc_loss, mag_loss = stft_loss(tf.expand_dims(y[:,0], 0), tf.expand_dims(logits[:,0], 0))
loss = l1 + sc_loss + mag_loss
loss = tf.reduce_mean(loss)
optimizer = tf.train.AdamOptimizer(learning_rate = 1e-3).minimize(loss)

In [24]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [25]:
# actual = (actual - np.mean(actual)) / np.std(actual)
# combined = (combined - np.mean(combined)) / np.std(combined)

In [26]:
%%time

sess.run([loss, optimizer], feed_dict = {i: np.expand_dims(combined, axis = -1),
                                                   y: np.expand_dims(actual, axis = -1)})

CPU times: user 5.4 s, sys: 1.57 s, total: 6.97 s
Wall time: 6.61 s


[0.9521234, None]

In [27]:
for e in range(3000):
    l, _ = sess.run([loss, optimizer], feed_dict = {i: np.expand_dims(combined, axis = -1),
                                                   y: np.expand_dims(actual, axis = -1)})
    if e % 10 == 0:
        print(e, l)

0 0.91275305
10 0.5970249
20 0.5196016
30 0.49426943
40 0.47991842
50 0.473474
60 0.46770263
70 0.4812496
80 0.47520447
90 0.45936626
100 0.45319855
110 0.44998062
120 0.46974474
130 0.45780542
140 0.46741617
150 0.45673484
160 0.4437884
170 0.4382532
180 0.4322368
190 0.42478633
200 0.42002192
210 0.41035032
220 0.41686058
230 0.4083769
240 0.4055273
250 0.3973133
260 0.39053172
270 0.39212823
280 0.38210297
290 0.37560564
300 0.4014154
310 0.38847256
320 0.3770736
330 0.3694521
340 0.36170223
350 0.35687375
360 0.37433606
370 0.3662179
380 0.3495065
390 0.34140968
400 0.3331293
410 0.33615828
420 0.32600117
430 0.32510138
440 0.31787074
450 0.31557697
460 0.31285685
470 0.31563485
480 0.3121789
490 0.304838
500 0.3019477
510 0.30612874
520 0.30200493
530 0.2984059
540 0.29564607
550 0.29315025
560 0.28991264
570 0.28630054
580 0.4588474
590 0.43183726
600 0.4035662
610 0.42969
620 0.40911037
630 0.3923209
640 0.39364684
650 0.38134062
660 0.38816315
670 0.37779826
680 0.36954266
690 

In [28]:
y_ = sess.run(logits, feed_dict = {i: np.expand_dims(combined, axis = -1)})

In [29]:
ipd.Audio(actual[:sr*5], rate = sr)

In [30]:
ipd.Audio(combined[:sr*5], rate = sr)

In [32]:
ipd.Audio(y_[:,0][:sr*5], rate = sr)

In [33]:
np.abs(actual - y_[:,0]).mean()

0.012030631366219279